In [1]:
import pandas as pd
import numpy as np
from matrixprofile import *

%matplotlib inline
import matplotlib.pyplot as plt
from saxpy.sax import sax_via_window
from sklearn.preprocessing import LabelEncoder

In [2]:
import numpy as np
from saxpy.znorm import znorm
from saxpy.sax import ts_to_string
from saxpy.alphabet import cuts_for_asize

In [3]:
def get_integer_mapping(le):
    '''
    Return a dict mapping labels to their integer values
    from an SKlearn LabelEncoder
    le = a fitted SKlearn LabelEncoder
    '''
    res = {}
    for cl in le.classes_:
        res.update({cl:le.transform([cl])[0]})

    return res

In [34]:
def get_patterns(df, size = 5):
    df['category2'][df['category2']=='communal'] = 'z'
    df['category'] = df['category2'].apply(lambda x: x[0])
    users1 = []
    actions1 = []
    users = np.array(df.userID.tolist())
    sessions = np.array(df.session.tolist())
    for i in range(1,len(users)):
        if users[i] != users[i-1]:
            actions1.append('stop')
            users1.append(users1[-1])
        elif sessions[i] != sessions[i-1]:
            actions1.append('break')
            users1.append(users1[-1])
        actions1.append(df.category[i])
        users1.append(users[i])
    actions_new = pd.DataFrame({'userID':users1, 'actions':actions1})
    
    
    my_list = []
    for name, groups in actions_new.groupby('userID'):
        user_list = [['0',1]]
        cnt_list = 0
        category = groups.actions.tolist()
        for i in range(0,len(category)):
            if category[i] == user_list[cnt_list][0]:
                user_list[cnt_list][1]+=1
            else:
                user_list.append([category[i],1])
                cnt_list+=1
        my_list.append(user_list[1:])
    for i in range(0,len(my_list)):
        my_list[i] = ["".join([str(k) for k in j]) for j in my_list[i]]
    users = []
    for names,groups in df.groupby('userID'):
        users.append(names)
    actions = []
    for i in my_list:
        actions.append(",".join(i))
    #df_temp = pd.DataFrame({'userID':users, 'actions':actions})
    flat_list_df = [item for sublist in actions for item in sublist.split(",")]
    lbl = LabelEncoder() 
    lbl.fit(flat_list_df)
    actions_lbl = [lbl.transform(i.split(",")) for i in actions]
    flat_lbls_df = [item for sublist in actions_lbl for item in sublist]
    sax_df = sax_via_window(flat_lbls_df,size,size,size,"none", 0.01)
    return [sax_df,flat_list_df]

In [32]:
def most_common(sax, cnt_lvl = 100, size = 5):
    patterns_dict = {}
    for pattern, cnt in sax[0].items():
        if len(cnt)>cnt_lvl:
            print(pattern+":"+(",".join([str(i) for i in cnt])))
            print()
            patterns_dict.update({pattern:cnt})
    for key,value in patterns_dict.items():
        print(key)
        print(sax[1][value[0]:value[0]+size])
        print(sax[1][value[5]:value[5]+size])
        print(sax[1][value[10]:value[10]+size])
        print()
    return patterns_dict

In [10]:
super_4 = pd.read_csv('./superusers_lvl4.csv')
other_4 = pd.read_csv('./other_lvl4.csv')
s4 = super_4[['userID','session','category2']]
o4 = other_4[['userID','session','category2']]

C:\Users\mistr\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [58]:
s4.shape

(265693, 4)

In [29]:
sax_s4 = get_patterns(s4,5)

C:\Users\mistr\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\mistr\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [30]:
sax_s4[0]

defaultdict(list,
            {'baeae': [0,
              40,
              127,
              213,
              249,
              269,
              343,
              375,
              395,
              407,
              489,
              501,
              535,
              541,
              569,
              585,
              607,
              631,
              649,
              817,
              853,
              973,
              1043,
              1065,
              1095,
              1101,
              1155,
              1161,
              1167,
              1371,
              1379,
              1455,
              1549,
              1577,
              1581,
              1597,
              1657,
              1683,
              1695,
              1723,
              1729,
              1865,
              1897,
              1923,
              2207,
              2295,
              2469,
              2483,
              2567,
              2613

In [33]:
most_common(sax_s4,100)

baeae:0,40,127,213,249,269,343,375,395,407,489,501,535,541,569,585,607,631,649,817,853,973,1043,1065,1095,1101,1155,1161,1167,1371,1379,1455,1549,1577,1581,1597,1657,1683,1695,1723,1729,1865,1897,1923,2207,2295,2469,2483,2567,2613,2681,2721,2795,2809,2819,2871,2891,2929,2935,2943,2973,3007,3017,3041,3071,3077,3153,3165,3187,3273,3301,3347,3395,3409,3423,3495,3509,3535,3583,3599,3645,3661,3707,3821,3883,4005,4047,4067,4097,4133,4139,4161,4203,4291,4317,4359,4367,4379,4437,4469,4523,4553,4619,4773,4801,4849,4863,4947,4955,4963,5069,5111,5213,5241,5259,5285,5299,5359,5367,5387,5455,5461,5497,5515,5551,5561,5581,5723,5835,5945,5969,6103,6115,6121,6137,6145,6197,6237,6263,6305,6315,6353,6383,6427,6533,6555,6581,6589,6639,6645,6699,6725,6863,6877,6895,6927,6981,6987,7003,7061,7101,7115,7147,7187,7237,7269,7357,7475,7519,7537,7549,7567,7579,7607,7657,7717,7767,7819,7845,7855,7879,7899,7927,7941,7963,7985,8041,8053,8089,8143,8179,8203,8239,8277,8303,8383,8393,8411,8487,8503,8513,8561,8571,8623

{'baeae': [0,
  40,
  127,
  213,
  249,
  269,
  343,
  375,
  395,
  407,
  489,
  501,
  535,
  541,
  569,
  585,
  607,
  631,
  649,
  817,
  853,
  973,
  1043,
  1065,
  1095,
  1101,
  1155,
  1161,
  1167,
  1371,
  1379,
  1455,
  1549,
  1577,
  1581,
  1597,
  1657,
  1683,
  1695,
  1723,
  1729,
  1865,
  1897,
  1923,
  2207,
  2295,
  2469,
  2483,
  2567,
  2613,
  2681,
  2721,
  2795,
  2809,
  2819,
  2871,
  2891,
  2929,
  2935,
  2943,
  2973,
  3007,
  3017,
  3041,
  3071,
  3077,
  3153,
  3165,
  3187,
  3273,
  3301,
  3347,
  3395,
  3409,
  3423,
  3495,
  3509,
  3535,
  3583,
  3599,
  3645,
  3661,
  3707,
  3821,
  3883,
  4005,
  4047,
  4067,
  4097,
  4133,
  4139,
  4161,
  4203,
  4291,
  4317,
  4359,
  4367,
  4379,
  4437,
  4469,
  4523,
  4553,
  4619,
  4773,
  4801,
  4849,
  4863,
  4947,
  4955,
  4963,
  5069,
  5111,
  5213,
  5241,
  5259,
  5285,
  5299,
  5359,
  5367,
  5387,
  5455,
  5461,
  5497,
  5515,
  5551,
  5561,
  5581,


In [35]:
sax_o4 = get_patterns(o4)

C:\Users\mistr\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\mistr\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [36]:
sax_o4[0]

defaultdict(list,
            {'eaead': [0,
              50,
              428,
              452,
              462,
              782,
              966,
              990,
              1172,
              1212,
              1368,
              1540,
              1544,
              1562,
              1586,
              1634,
              1646,
              2024,
              2166,
              2434,
              2454,
              2462,
              2476,
              2594,
              2628,
              2664,
              2742,
              3380,
              3382,
              3690,
              3692,
              3726,
              3740,
              3780,
              3912,
              3998,
              4004,
              4104,
              4164,
              4714,
              5074,
              5252,
              5280,
              5288,
              5342,
              5478,
              5594,
              5634,
              5848,
    

In [37]:
most_common(sax_o4)

eaead:0,50,428,452,462,782,966,990,1172,1212,1368,1540,1544,1562,1586,1634,1646,2024,2166,2434,2454,2462,2476,2594,2628,2664,2742,3380,3382,3690,3692,3726,3740,3780,3912,3998,4004,4104,4164,4714,5074,5252,5280,5288,5342,5478,5594,5634,5848,5926,6012,6030,6346,6352,6412,6522,6626,6814,7022,7210,7450,7536,7544,7598,7674,7902,7932,7980,8124,8302,8448,8874,8992,9172,9198,9256,9336,9451,9519,9651,9705,9719,9781,9805,9961,10055,10273,10537,10581,10743,10817,10909,11403,11999,12073,12153,12183,12217,12273,12471,12701,12883,12926,13084,13134,13152,13168,13360,13396,13478,13678,13734,13758,14284,14286,14318,14380,14412,14432,14480,14526,14588,14804,14880,14916,15188,15338,16146,16440,17004,17078,17268,17328,17604,17684,17710,17798,17858,18086,18122,18176,18360,18362,18366,18414,18488,18888,18910,19480,19482,19660,19726,19758,19810,19890,19896,19982,19994,20068,20346,20540,20612,20834,20852,20904,20910,20950,20958,20962,21032,21064,21070,21360,21374,21592,21612,21666,21774,22102,22148,22252,2252

{'eaead': [0,
  50,
  428,
  452,
  462,
  782,
  966,
  990,
  1172,
  1212,
  1368,
  1540,
  1544,
  1562,
  1586,
  1634,
  1646,
  2024,
  2166,
  2434,
  2454,
  2462,
  2476,
  2594,
  2628,
  2664,
  2742,
  3380,
  3382,
  3690,
  3692,
  3726,
  3740,
  3780,
  3912,
  3998,
  4004,
  4104,
  4164,
  4714,
  5074,
  5252,
  5280,
  5288,
  5342,
  5478,
  5594,
  5634,
  5848,
  5926,
  6012,
  6030,
  6346,
  6352,
  6412,
  6522,
  6626,
  6814,
  7022,
  7210,
  7450,
  7536,
  7544,
  7598,
  7674,
  7902,
  7932,
  7980,
  8124,
  8302,
  8448,
  8874,
  8992,
  9172,
  9198,
  9256,
  9336,
  9451,
  9519,
  9651,
  9705,
  9719,
  9781,
  9805,
  9961,
  10055,
  10273,
  10537,
  10581,
  10743,
  10817,
  10909,
  11403,
  11999,
  12073,
  12153,
  12183,
  12217,
  12273,
  12471,
  12701,
  12883,
  12926,
  13084,
  13134,
  13152,
  13168,
  13360,
  13396,
  13478,
  13678,
  13734,
  13758,
  14284,
  14286,
  14318,
  14380,
  14412,
  14432,
  14480,
  14526

In [38]:
super_3 = pd.read_csv('./superusers_lvl3.csv')
other_3 = pd.read_csv('./other_lvl3.csv')
s3 = super_3[['userID','session','category2']]
o3 = other_3[['userID','session','category2']]

C:\Users\mistr\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (7,8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [39]:
sax_s3 = get_patterns(s3,5)

C:\Users\mistr\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\mistr\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [41]:
most_common(sax_s3, 50)

bdbeb:1,14,22,24,58,70,90,184,216,250,334,338,354,356,358,414,438,460,478,542,548,554,584,614,628,636,642,672,688,702,724,732,734,746,762,778,820,836,848,862,872,886,896,900,906,910,934,986,998,1014,1018,1024,1034,1042,1068,1070,1072,1086,1108,1120,1138,1144,1164,1166,1196,1206,1250,1264,1278,1296,1312,1314,1346,1376,1386,1388,1426,1492

bcbeb:3,12,18,46,54,62,104,122,146,222,256,294,318,322,340,396,434,442,558,564,582,604,610,632,640,656,692,696,708,730,760,786,796,818,876,882,890,952,978,988,990,1006,1012,1026,1048,1054,1078,1100,1104,1116,1156,1214,1220,1226,1232,1242,1254,1262,1274,1318,1324,1402,1420,1424,1450,1472,1502

bebdb:5,20,48,50,56,64,88,124,130,132,172,182,192,214,220,240,264,278,292,320,360,394,416,432,440,444,476,536,546,552,562,598,602,612,624,630,644,658,670,686,694,700,706,712,790,794,800,806,814,826,832,838,850,880,898,908,912,914,922,932,938,960,974,982,1000,1002,1032,1036,1040,1050,1052,1084,1094,1102,1118,1128,1136,1154,1162,1184,1190,1200,1216,1222,1248,1258,12

{'bdbeb': [1,
  14,
  22,
  24,
  58,
  70,
  90,
  184,
  216,
  250,
  334,
  338,
  354,
  356,
  358,
  414,
  438,
  460,
  478,
  542,
  548,
  554,
  584,
  614,
  628,
  636,
  642,
  672,
  688,
  702,
  724,
  732,
  734,
  746,
  762,
  778,
  820,
  836,
  848,
  862,
  872,
  886,
  896,
  900,
  906,
  910,
  934,
  986,
  998,
  1014,
  1018,
  1024,
  1034,
  1042,
  1068,
  1070,
  1072,
  1086,
  1108,
  1120,
  1138,
  1144,
  1164,
  1166,
  1196,
  1206,
  1250,
  1264,
  1278,
  1296,
  1312,
  1314,
  1346,
  1376,
  1386,
  1388,
  1426,
  1492],
 'bcbeb': [3,
  12,
  18,
  46,
  54,
  62,
  104,
  122,
  146,
  222,
  256,
  294,
  318,
  322,
  340,
  396,
  434,
  442,
  558,
  564,
  582,
  604,
  610,
  632,
  640,
  656,
  692,
  696,
  708,
  730,
  760,
  786,
  796,
  818,
  876,
  882,
  890,
  952,
  978,
  988,
  990,
  1006,
  1012,
  1026,
  1048,
  1054,
  1078,
  1100,
  1104,
  1116,
  1156,
  1214,
  1220,
  1226,
  1232,
  1242,
  1254,
  1262

In [42]:
sax_o3 = get_patterns(o3)

C:\Users\mistr\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\mistr\Anaconda3\lib\site-packages\pandas\core\generic.py:8766: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)
C:\Users\mistr\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This

In [43]:
most_common(sax_o3)

bbcbe:6,144,230,236,254,256,492,552,570,608,812,882,932,1284,1330,1540,1618,1748,1876,1922,1982,2040,2338,2404,2750,2824,3086,3108,3148,3162,3388,3394,3466,3970,4082,4366,4444,4594,4602,4612,4662,5078,5193,5361,5459,5691,5737,5959,5995,6641,7135,7167,7413,7773,7779,7913,7935,7985,7993,8035,8213,8285,8349,8507,8525,8687,8721,8883,9067,9183,9281,9663,10231,10371,10421,10767,10867,10921,11537,11663,11813,11993,12189,12409,12449,12855,13015,13205,13343,13535,13551,13561,13681,13809,13971,14088,15150,15328,15800,16012,16188,16206,16234,17086,17356,17514,17616

adeda:15,97,441,469,649,793,1053,1071,1339,2145,2171,2503,2609,2801,3067,3347,3571,3667,3739,3801,4211,4433,4497,4627,5104,5108,5160,5330,5504,5772,5838,6020,6302,6336,6536,6658,6936,7034,7138,8398,8442,8608,8736,8740,8990,9214,9710,9782,10056,10094,10358,10480,11184,11298,11486,11658,11666,12000,12218,12236,12290,12306,12326,12386,12700,12722,12734,13110,13148,13212,13448,13524,13812,13984,14105,14119,14423,14581,14683,14823,15027,15

{'bbcbe': [6,
  144,
  230,
  236,
  254,
  256,
  492,
  552,
  570,
  608,
  812,
  882,
  932,
  1284,
  1330,
  1540,
  1618,
  1748,
  1876,
  1922,
  1982,
  2040,
  2338,
  2404,
  2750,
  2824,
  3086,
  3108,
  3148,
  3162,
  3388,
  3394,
  3466,
  3970,
  4082,
  4366,
  4444,
  4594,
  4602,
  4612,
  4662,
  5078,
  5193,
  5361,
  5459,
  5691,
  5737,
  5959,
  5995,
  6641,
  7135,
  7167,
  7413,
  7773,
  7779,
  7913,
  7935,
  7985,
  7993,
  8035,
  8213,
  8285,
  8349,
  8507,
  8525,
  8687,
  8721,
  8883,
  9067,
  9183,
  9281,
  9663,
  10231,
  10371,
  10421,
  10767,
  10867,
  10921,
  11537,
  11663,
  11813,
  11993,
  12189,
  12409,
  12449,
  12855,
  13015,
  13205,
  13343,
  13535,
  13551,
  13561,
  13681,
  13809,
  13971,
  14088,
  15150,
  15328,
  15800,
  16012,
  16188,
  16206,
  16234,
  17086,
  17356,
  17514,
  17616],
 'adeda': [15,
  97,
  441,
  469,
  649,
  793,
  1053,
  1071,
  1339,
  2145,
  2171,
  2503,
  2609,
  2801,
 

In [44]:
super_2 = pd.read_csv('./superusers_lvl2.csv')
other_2 = pd.read_csv('./other_lvl2.csv')
s2 = super_2[['userID','session','category2']]
o2 = other_2[['userID','session','category2']]

C:\Users\mistr\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (7,8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [45]:
sax_s2 = get_patterns(s2)

C:\Users\mistr\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\mistr\Anaconda3\lib\site-packages\pandas\core\generic.py:8766: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)
C:\Users\mistr\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This

In [47]:
most_common(sax_s2,50)

bebeb:2,28,30,40,50,60,68,70,72,74,78,100,128,158,218,220,234,260,264,266,276,318,326,338,340,358,366,370,376,380,382,388,394,428,448,450,452,464,472,484,498,500,502,508,510,512,514,516,518,524,526,534,540,544,546,558,568

bebeb
['break1', 'c53', 'break1', 'c8', 'break1']
['break1', 'c8', 'break1', 'c7', 'break1']
['break1', 'c2', 'break1', 'c2', 'break1']



{'bebeb': [2,
  28,
  30,
  40,
  50,
  60,
  68,
  70,
  72,
  74,
  78,
  100,
  128,
  158,
  218,
  220,
  234,
  260,
  264,
  266,
  276,
  318,
  326,
  338,
  340,
  358,
  366,
  370,
  376,
  380,
  382,
  388,
  394,
  428,
  448,
  450,
  452,
  464,
  472,
  484,
  498,
  500,
  502,
  508,
  510,
  512,
  514,
  516,
  518,
  524,
  526,
  534,
  540,
  544,
  546,
  558,
  568]}

In [48]:
sax_o2 = get_patterns(o2)

C:\Users\mistr\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\mistr\Anaconda3\lib\site-packages\pandas\core\generic.py:8766: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)
C:\Users\mistr\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This

In [49]:
most_common(sax_o2)

bebdb:7,9,11,57,61,119,147,235,237,339,347,359,403,419,427,453,583,795,973,977,1197,1199,1307,1455,1459,1477,1703,1931,1949,1957,2089,2171,2173,2193,2205,2207,2241,2663,3023,3281,3283,3713,3731,3947,3955,4003,4096,4360,4592,4684,4714,4896,5056,5130,5216,5516,5518,5784,5850,6042,6200,6442,6444,6526,6658,6728,6736,6992,7002,7020,7032,7078,7110,7672,7776,7782,8012,8298,8546,8628,8722,9004,9386,9476,9870,9872,10248,11148,11292,11294,11370,11374,11398,11440,11638,11658,11768,12072,12280,12282,12286,12290,12798,13352,13386,13590,13618,14240,14254,14522,14528,14644,14648,14670,14792,14794,15264,15302,15310,15318,15896,16012,16014

aedea:22,76,80,1100,1122,1126,1290,1314,1356,1434,1440,1802,2014,2036,2118,2144,2424,2490,2522,2608,2774,2866,2880,2974,2984,3004,3062,3098,3112,3202,3326,3330,3354,3392,3448,3546,3564,3626,3912,4145,4317,4385,4479,4503,4541,4703,4765,4845,4919,4979,5081,5555,5559,5583,5857,5867,5945,5981,6387,6409,6493,6543,6601,6723,6811,7055,7129,7269,7315,7319,7325,7337,7811,797

{'bebdb': [7,
  9,
  11,
  57,
  61,
  119,
  147,
  235,
  237,
  339,
  347,
  359,
  403,
  419,
  427,
  453,
  583,
  795,
  973,
  977,
  1197,
  1199,
  1307,
  1455,
  1459,
  1477,
  1703,
  1931,
  1949,
  1957,
  2089,
  2171,
  2173,
  2193,
  2205,
  2207,
  2241,
  2663,
  3023,
  3281,
  3283,
  3713,
  3731,
  3947,
  3955,
  4003,
  4096,
  4360,
  4592,
  4684,
  4714,
  4896,
  5056,
  5130,
  5216,
  5516,
  5518,
  5784,
  5850,
  6042,
  6200,
  6442,
  6444,
  6526,
  6658,
  6728,
  6736,
  6992,
  7002,
  7020,
  7032,
  7078,
  7110,
  7672,
  7776,
  7782,
  8012,
  8298,
  8546,
  8628,
  8722,
  9004,
  9386,
  9476,
  9870,
  9872,
  10248,
  11148,
  11292,
  11294,
  11370,
  11374,
  11398,
  11440,
  11638,
  11658,
  11768,
  12072,
  12280,
  12282,
  12286,
  12290,
  12798,
  13352,
  13386,
  13590,
  13618,
  14240,
  14254,
  14522,
  14528,
  14644,
  14648,
  14670,
  14792,
  14794,
  15264,
  15302,
  15310,
  15318,
  15896,
  16012,
  1601

In [50]:
super_1 = pd.read_csv('./superusers_lvl1.csv')
other_1 = pd.read_csv('./other_lvl1.csv')
s1 = super_1[['userID','session','category2']]
o1 = other_1[['userID','session','category2']]

C:\Users\mistr\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [51]:
sax_s1 = get_patterns(s1)

C:\Users\mistr\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\mistr\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [55]:
most_common(sax_s1,30)

bebbb:4,30,40,44,50,56,60,80,92,96,104,182,186,278,294,304,342,374,378,382,402,434,440,508,512,534,544,560,574,600,612,624

bbbeb:6,32,42,52,58,62,72,88,94,98,142,168,184,188,280,306,316,350,376,380,384,404,442,510,514,536,546,564,576,582,598,604

bebeb:8,28,38,64,74,76,82,84,86,90,100,112,118,174,176,180,190,192,200,224,249,282,284,292,300,386,392,394,406,412,416,424,426,444,448,450,452,462,464,474,476,480,484,518,528,530,532,540,542,548,552,562,566,584,588,590,606,608,610,636,652

bebbb
['break1', 'c20', 'break1', 'c1', 'break1']
['break1', 'c4', 'break1', 'c10', 'break1']
['break1', 'c8', 'break1', 'c1', 'break1']

bbbeb
['break1', 'c1', 'break1', 'c20', 'break1']
['break1', 'c115', 'break1', 'c7', 'break1']
['break1', 'c1', 'break1', 'c37', 'break1']

bebeb
['break1', 'c20', 'break1', 'c24', 'break1']
['break1', 'c5', 'break1', 'c5', 'break1']
['break1', 'c2', 'break1', 'c2', 'break1']



{'bebbb': [4,
  30,
  40,
  44,
  50,
  56,
  60,
  80,
  92,
  96,
  104,
  182,
  186,
  278,
  294,
  304,
  342,
  374,
  378,
  382,
  402,
  434,
  440,
  508,
  512,
  534,
  544,
  560,
  574,
  600,
  612,
  624],
 'bbbeb': [6,
  32,
  42,
  52,
  58,
  62,
  72,
  88,
  94,
  98,
  142,
  168,
  184,
  188,
  280,
  306,
  316,
  350,
  376,
  380,
  384,
  404,
  442,
  510,
  514,
  536,
  546,
  564,
  576,
  582,
  598,
  604],
 'bebeb': [8,
  28,
  38,
  64,
  74,
  76,
  82,
  84,
  86,
  90,
  100,
  112,
  118,
  174,
  176,
  180,
  190,
  192,
  200,
  224,
  249,
  282,
  284,
  292,
  300,
  386,
  392,
  394,
  406,
  412,
  416,
  424,
  426,
  444,
  448,
  450,
  452,
  462,
  464,
  474,
  476,
  480,
  484,
  518,
  528,
  530,
  532,
  540,
  542,
  548,
  552,
  562,
  566,
  584,
  588,
  590,
  606,
  608,
  610,
  636,
  652]}

In [56]:
sax_o1 = get_patterns(o1)

C:\Users\mistr\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\mistr\Anaconda3\lib\site-packages\pandas\core\generic.py:8766: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)
C:\Users\mistr\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This

In [57]:
most_common(sax_o1)

bedea:0,374,704,724,780,1222,1558,1580,1684,1852,1952,2004,2210,2700,2806,2932,3224,3578,3698,3712,3732,3948,4230,4380,4466,4694,4764,5183,5313,5425,5515,6203,6235,6515,7135,7539,8573,8655,8729,8819,8877,8895,9071,9131,9141,9411,9597,10061,10265,10523,10623,10691,10919,10967,11319,11519,11955,12313,12343,12381,12389,12525,12591,12621,13107,13241,13309,13415,13449,13551,13695,13885,13935,14149,14269,14423,14545,14625,14825,14903,14993,15105,15435,15485,15605,16091,16175,16535,16697,16801,16917,17411,17479,17677,17703,18361,19522,19740,19992,20346,20418,20478,20686,20690,20718,21221,21311,21397,21769,21891,21915,22203,22275,22561,23085,23369,23517,23627,23765,23969,24105,24369,24431,24483

dcdad:1,135,205,241,265,273,303,375,393,423,453,545,567,613,667,695,705,725,729,755,781,937,1065,1109,1145,1159,1171,1195,1197,1223,1285,1385,1403,1405,1443,1465,1559,1569,1581,1611,1673,1705,1765,1835,1897,1909,1937,1953,1963,2005,2027,2061,2085,2095,2099,2127,2139,2147,2193,2205,2281,2297,2309,2311,2

{'bedea': [0,
  374,
  704,
  724,
  780,
  1222,
  1558,
  1580,
  1684,
  1852,
  1952,
  2004,
  2210,
  2700,
  2806,
  2932,
  3224,
  3578,
  3698,
  3712,
  3732,
  3948,
  4230,
  4380,
  4466,
  4694,
  4764,
  5183,
  5313,
  5425,
  5515,
  6203,
  6235,
  6515,
  7135,
  7539,
  8573,
  8655,
  8729,
  8819,
  8877,
  8895,
  9071,
  9131,
  9141,
  9411,
  9597,
  10061,
  10265,
  10523,
  10623,
  10691,
  10919,
  10967,
  11319,
  11519,
  11955,
  12313,
  12343,
  12381,
  12389,
  12525,
  12591,
  12621,
  13107,
  13241,
  13309,
  13415,
  13449,
  13551,
  13695,
  13885,
  13935,
  14149,
  14269,
  14423,
  14545,
  14625,
  14825,
  14903,
  14993,
  15105,
  15435,
  15485,
  15605,
  16091,
  16175,
  16535,
  16697,
  16801,
  16917,
  17411,
  17479,
  17677,
  17703,
  18361,
  19522,
  19740,
  19992,
  20346,
  20418,
  20478,
  20686,
  20690,
  20718,
  21221,
  21311,
  21397,
  21769,
  21891,
  21915,
  22203,
  22275,
  22561,
  23085,
  23369,
 